In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [99]:
%matplotlib inline

In [101]:
# read covid data taken from the WHO and population data 2020

In [102]:
data = pd.read_csv('C:/Users/shaha/OneDrive/Documents/data/covid/WHO-COVID-19-global-data.csv')

In [103]:
pop = pd.read_csv('C:/Users/shaha/OneDrive/Documents/data/population_by_country_2020.csv')

In [104]:
data.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0


In [105]:
# edit the dataframes

In [106]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61805 entries, 0 to 61804
Data columns (total 8 columns):
Date_reported         61805 non-null object
 Country_code         61542 non-null object
 Country              61805 non-null object
 WHO_region           61805 non-null object
 New_cases            61805 non-null int64
 Cumulative_cases     61805 non-null int64
 New_deaths           61805 non-null int64
 Cumulative_deaths    61805 non-null int64
dtypes: int64(4), object(4)
memory usage: 3.8+ MB


In [107]:
#remove WHO region
data.drop(data.columns[3], axis=1, inplace=True)

In [108]:
# rename columns
data.rename(columns={'Date_reported': 'Date' ,' Country_code': 'Code', ' New_cases' : 'NCases',
                     ' Cumulative_cases': 'CCases',' New_deaths': 'NDeaths',' Cumulative_deaths': 'CDeaths',
                     ' Country': 'Country'},errors="raise", inplace=True)

In [109]:
#change date to datetime
data['Date'] = pd.to_datetime(data['Date'])

In [110]:
data.head()

,Date,Code,Country,NCases,CCases,NDeaths,CDeaths
0,2020-01-03,AF,Afghanistan,0,0,0,0
1,2020-01-04,AF,Afghanistan,0,0,0,0
2,2020-01-05,AF,Afghanistan,0,0,0,0
3,2020-01-06,AF,Afghanistan,0,0,0,0
4,2020-01-07,AF,Afghanistan,0,0,0,0


In [111]:
# replace Mainland China with China
data['Country']=data['Country'].replace('Mainland China', 'China')

In [112]:
# Most recent recorded data - 21st September

In [113]:
data_sept21 = data[data['Date'] == max(data['Date'])].reset_index().groupby('Date')['CCases', 'CDeaths'].sum().reset_index()

In [114]:
data_sept21.head()

,Date,CCases,CDeaths
0,2020-09-21,30949804,959116


In [115]:
data_sept21['Active/Recovered'] = data_sept21['CCases']-data_sept21['CDeaths']

In [116]:
data_sept21.head()

,Date,CCases,CDeaths,Active/Recovered
0,2020-09-21,30949804,959116,29990688


In [117]:
fig = px.pie(values =data_sept21.loc[0, ["CDeaths","Active/Recovered"]], names=['Deaths', 'Active/Recovereed'], 
                             title='Global Covid 19 figures, to date 21st September', hole = 0.7,
                            color_discrete_sequence = px.colors.qualitative.Antique )
fig.show()

In [118]:
# Total Global Covid Cases over time. Displayed as Bar Graph

In [119]:
data_bydate = data.groupby('Date').sum().reset_index()

In [120]:
fig = px.bar(data_bydate, x='Date', y='CCases', color='CDeaths',
             width=800,labels={'CCases': 'Total Cases', 'CDeaths': 'Deaths'},
            title = 'Global figures for Covid-19 over time')

fig.show()

In [121]:
# Total Global Covid Cases over time. Displayed as Scatter Plot

In [122]:
fig = px.scatter(data_bydate, x='Date', y='CCases', color='CDeaths',
             width=800,labels={'CCases': 'Total Cases', 'CDeaths': 'Deaths'},
            title = 'Global figures for Covid-19 over time')

fig.show()

In [123]:
# Total global cases and deaths in May

In [124]:
data_bydate_may = data_bydate[data_bydate['Date'].dt.month == 5]
fig = px.scatter(data_bydate_may, x='Date', y='CCases', color='CDeaths',
             width=800,labels={'CCases': 'Total Cases', 'CDeaths': 'Deaths'},
            title = 'Global figures for Covid-19 in May')

fig.show()

In [125]:
# Worldwide Confirmed cases & Deaths, as a line graph

In [126]:
fig = px.line(data_bydate, x='Date', y=['CCases','CDeaths'], 
                            color_discrete_sequence = px.colors.qualitative.Antique,
                           title = 'Global Confirmed cases and Deaths over time')
fig.show()

In [127]:
# Top 10 countires with highest Confirmed cases  and highest death rates Worldwide

In [128]:
data_highest_countries = data.groupby('Country')['NCases', 'NDeaths'].sum().reset_index().sort_values(
        by = 'NCases', ascending = False).head(10).reset_index()

In [129]:
data_highest_countries

,index,Country,NCases,NDeaths
0,224,United States of America,6703698,198094
1,96,India,5487580,87882
2,28,Brazil,4528240,136532
3,173,Russian Federation,1109595,19489
4,163,Peru,762865,31369
5,43,Colombia,758398,24039
6,134,Mexico,694121,73258
7,197,South Africa,661211,15953
8,199,Spain,640040,30495
9,8,Argentina,622934,12909


In [130]:
fig = px.bar(data_highest_countries, x='Country', y=['NCases', 'NDeaths'], barmode='overlay', opacity=0.4,
             title = 'Covid Cases and Deaths for the Countries with the 10 highest Cases in the world' )
fig.show()

In [131]:
# select full set of data by date for only the countries with the 10 highest confirmed cases - Sort by country

In [132]:
totaldata_highest_countries = data.loc[data.Country.isin(data_highest_countries.Country)]

In [133]:
fig = px.line(totaldata_highest_countries, x='Date', y='CCases', color = 'Country', 
              labels={'CCases': 'Confirmed Cases'},
              title = 'Covid 19 Confirmed Cases in Top 10 countries')
fig.show()

In [134]:
fig = px.line(totaldata_highest_countries, x='Date', y='CDeaths', color = 'Country', 
              labels={'CDeaths': 'Confirmed Deaths'},
              title = 'Covid 19 Confirmed Deaths in Top 10 countries')
fig.show()

In [135]:
# Daily Cases figures for 10 Countries with highest cases

In [136]:
fig= px.line(totaldata_highest_countries, x='Date', y=['NCases'], color = 'Country', 
                           title = 'Covid 19 New Daily Cases in Top 10 countries')
fig.show()

In [137]:
fig = px.line(totaldata_highest_countries, x='Date', y=['NDeaths'], color = 'Country', 
                           title = 'Covid 19 New Daily Deaths in Top 10 countries')
fig.show()

In [138]:
fig = px.bar(data_bydate, x='Date', y='NCases', color='NDeaths',
             width=800,labels={'NCases': 'New Cases', 'NDeaths': 'New Deaths'},
            title = 'Global figures of new daily cases for Covid 19')

fig.show()

In [139]:
# Figures for individual countires

In [140]:
UK_data = data[data['Country']=='The United Kingdom']

In [141]:
fig = px.line(UK_data, x='Date', y=['NCases','NDeaths'], 
                            color_discrete_sequence = px.colors.qualitative.Antique,
                           title = 'UK Confirmed cases and Deaths over time')
fig.show()

In [142]:
USA_data = data[data['Country']=='United States of America']

In [143]:
fig = px.line(USA_data, x='Date', y=['NCases','NDeaths'], 
                            color_discrete_sequence = px.colors.qualitative.Antique,
                           title = 'Confirmed cases and Deaths over time for the USA')
fig.show()

In [144]:
India_data = data[data['Country']=='India']

In [145]:
fig = px.line(India_data, x='Date', y=['NCases','NDeaths'], 
                            color_discrete_sequence = px.colors.qualitative.Antique,
                           title = 'Global Confirmed cases and Deaths over time fro India')
fig.show()

In [146]:
Russia_data = data[data['Country']=='Russian Federation']

In [147]:
fig = px.line(Russia_data, x='Date', y=['NCases','NDeaths'], 
                            color_discrete_sequence = px.colors.qualitative.Antique,
                           title = 'Global Confirmed cases and Deaths over time for Russia')
fig.show()

In [148]:
# How many unique country names in the population df 

In [149]:
pop['Country (or dependency)'].nunique()

235

In [150]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 11 columns):
Country (or dependency)    235 non-null object
Population (2020)          235 non-null int64
Yearly Change              235 non-null object
Net Change                 235 non-null int64
Density (P/Km²)            235 non-null int64
Land Area (Km²)            235 non-null int64
Migrants (net)             201 non-null float64
Fert. Rate                 235 non-null object
Med. Age                   235 non-null object
Urban Pop %                235 non-null object
World Share                235 non-null object
dtypes: float64(1), int64(4), object(6)
memory usage: 20.3+ KB


In [151]:
# How many unique country names in the covid 19 df 

In [152]:
data['Country'].nunique()

235

In [153]:
# edit the population df to remove unnecessary columns and change the column names for Country

In [154]:
pop.head()

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1440297825,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1382345085,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,274021604,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %


In [155]:
pop.drop(pop.columns[[2,3, 4, 5, 6, 7, 8, 9]], axis=1, inplace=True)

In [156]:
pop.head()

,Country (or dependency),Population (2020),World Share
0,China,1440297825,18.47 %
1,India,1382345085,17.70 %
2,United States,331341050,4.25 %
3,Indonesia,274021604,3.51 %
4,Pakistan,221612785,2.83 %


In [157]:
pop.rename(columns={'Country (or dependency)': 'Country' ,'Population (2020)': 'Population'},errors="raise", inplace=True)

In [158]:
pop.head()

,Country,Population,World Share
0,China,1440297825,18.47 %
1,India,1382345085,17.70 %
2,United States,331341050,4.25 %
3,Indonesia,274021604,3.51 %
4,Pakistan,221612785,2.83 %


In [159]:
# check how many country names match between the two dfs (199)

In [160]:
country_match = pop.loc[pop.Country.isin(data.Country)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 234
Data columns (total 3 columns):
Country        199 non-null object
Population     199 non-null int64
World Share    199 non-null object
dtypes: int64(1), object(2)
memory usage: 6.2+ KB


In [161]:
# find the countries that have that names that do not match in the two df's, using left / right excluding merges

In [162]:
data_excl = data.merge(pop, on='Country', how='left', indicator=True).query(
    '_merge == "left_only"').drop('_merge', 1).sort_values(by=['Country'])

In [163]:
pop_excl = data.merge(pop, on='Country', how='right', indicator=True).query(
    '_merge == "right_only"').drop('_merge', 1).sort_values(by=['Country'])

In [164]:
# find the names of all the countries in each df that do not match  

In [165]:
data_excl['Country'].unique()

array(['Bolivia (Plurinational State of)',
       'Bonaire, Sint Eustatius and Saba', 'Brunei Darussalam', 'Czechia',
       'Côte d’Ivoire', "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Falkland Islands (Malvinas)',
       'Faroe Islands', 'Guernsey', 'Iran (Islamic Republic of)',
       'Jersey', 'Kosovo[1]', "Lao People's Democratic Republic",
       'Micronesia (Federated States of)',
       'Northern Mariana Islands (Commonwealth of the)', 'Other',
       'Pitcairn Islands', 'Republic of Korea', 'Republic of Moldova',
       'Russian Federation', 'Saint Barthélemy', 'Saint Kitts and Nevis',
       'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines',
       'Sao Tome and Principe', 'Syrian Arab Republic',
       'The United Kingdom', 'Turks and Caicos Islands',
       'United Republic of Tanzania', 'United States Virgin Islands',
       'United States of America', 'Venezuela (Bolivarian Republic of)',
       'Viet Nam', 'Wallis an

In [166]:
pop_excl['Country'].unique()

array(['Bolivia', 'Brunei', 'Caribbean Netherlands', 'Channel Islands',
       'Czech Republic (Czechia)', "Côte d'Ivoire", 'DR Congo',
       'Faeroe Islands', 'Falkland Islands', 'Hong Kong', 'Iran', 'Laos',
       'Macao', 'Micronesia', 'Moldova', 'North Korea',
       'Northern Mariana Islands', 'Russia', 'Saint Barthelemy',
       'Saint Kitts & Nevis', 'Saint Pierre & Miquelon',
       'Sao Tome & Principe', 'South Korea', 'St. Vincent & Grenadines',
       'State of Palestine', 'Syria', 'Taiwan', 'Tanzania',
       'Turks and Caicos', 'U.S. Virgin Islands', 'United Kingdom',
       'United States', 'Venezuela', 'Vietnam', 'Wallis & Futuna',
       'Western Sahara'], dtype=object)

In [167]:
# manually replace names of countries so that they match in spelling, making sure that they refer to the same data

In [168]:
data['Country'] = data['Country'].replace({       
                    'Bolivia (Plurinational State of)': 'Bolivia' ,
                    'Brunei Darussalam': 'Brunei',
                    'Bonaire, Sint Eustatius and Saba': 'Caribbean Netherlands',
                    'Côte d’Ivoire':"Côte d'Ivoire",
                    'Czechia': 'Czech Republic (Czechia)', 
                    'Guernsey':'Channel Islands',
                    'Jersey':'Channel Islands',
                    'Democratic Republic of the Congo' : 'DR Congo',
                    "Democratic People's Republic of Korea" : 'North Korea',
                    'Republic of Korea': 'South Korea',
                    'Falkland Islands (Malvinas)': 'Falkland Islands',
                    'Faroe Islands': 'Faeroe Islands',
                    'Iran (Islamic Republic of)':'Iran',
                    "Lao People's Democratic Republic": 'Laos', 
                    'Northern Mariana Islands (Commonwealth of the)': 'Northern Mariana Islands',
                    'Micronesia (Federated States of)': 'Micronesia',
                    'occupied Palestinian territory, including east Jerusalem': 'State of Palestine',
                    'Republic of Moldova':'Moldova',
                    'Syrian Arab Republic' : 'Syria',
                    'Saint Pierre and Miquelon': 'Saint Pierre & Miquelon',
                    'Saint Vincent and the Grenadines': 'St. Vincent & Grenadines',
                    'Saint Kitts and Nevis':'Saint Kitts & Nevis', 
                    'Saint Barthélemy': 'Saint Barthelemy',
                    'Sao Tome and Principe':'Sao Tome & Principe',                    
                    'Russian Federation': 'Russia',
                    'United States of America': 'United States',
                    'The United Kingdom': 'United Kingdom',
                    'Wallis and Futuna': 'Wallis & Futuna',
                    'Viet Nam': 'Vietnam',
                    'Venezuela (Bolivarian Republic of)':'Venezuela', 
                    'Turks and Caicos Islands':'Turks and Caicos',
                    'United Republic of Tanzania':'Tanzania',
                    'United States Virgin Islands': 'U.S. Virgin Islands',
                    'Kosovo[1]': 'Other',
                    'Pitcairn Islands': 'Other'
    
                      })

In [169]:
# aggregate the values for the multiple rows for China

In [170]:
pop['Country'] = pop['Country'].replace({       
                    'Hong Kong': 'China',
                    'Macao': 'China',
                    'Taiwan': 'China',
                    'Western Sahara': 'Other'
                      })

In [171]:
pop.nunique()

Country        232
Population     235
World Share     74
dtype: int64

In [172]:
pop.head()

,Country,Population,World Share
0,China,1440297825,18.47 %
1,India,1382345085,17.70 %
2,United States,331341050,4.25 %
3,Indonesia,274021604,3.51 %
4,Pakistan,221612785,2.83 %


In [173]:
pop = pop.groupby('Country').sum().reset_index()

In [174]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 2 columns):
Country       232 non-null object
Population    232 non-null int64
dtypes: int64(1), object(1)
memory usage: 3.8+ KB


In [175]:
# now that all countries match - merge the dfs 

In [176]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61805 entries, 0 to 61804
Data columns (total 7 columns):
Date       61805 non-null datetime64[ns]
Code       61542 non-null object
Country    61805 non-null object
NCases     61805 non-null int64
CCases     61805 non-null int64
NDeaths    61805 non-null int64
CDeaths    61805 non-null int64
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 3.3+ MB


In [177]:
# df data group the total cases and deaths for each country total - divide that by population to get ratio

In [178]:
deaths_bycountry = data.groupby('Country')['NDeaths', 'NCases'].sum().reset_index().sort_values(by='NCases', ascending = False).reset_index()

In [179]:
deaths_bycountry.head()

,index,Country,NDeaths,NCases
0,222,United States,198094,6703698
1,95,India,87882,5487580
2,27,Brazil,136532,4528240
3,169,Russia,19489,1109595
4,162,Peru,31369,762865


In [180]:
covid_by_pop = deaths_bycountry.merge(pop, on='Country')

In [181]:
covid_by_pop.head()

,index,Country,NDeaths,NCases,Population
0,222,United States,198094,6703698,331341050
1,95,India,87882,5487580,1382345085
2,27,Brazil,136532,4528240,212821986
3,169,Russia,19489,1109595,145945524
4,162,Peru,31369,762865,33050211


In [182]:
# calculate the deaths per capita per million

In [183]:
covid_by_pop['Cases per Capita'] = (covid_by_pop['NDeaths']/covid_by_pop['Population'])* 1000000

In [184]:
# remove the category other as it distorts the per capita results

In [185]:
covid_by_pop = covid_by_pop[covid_by_pop.Country != 'Other'].sort_values(by='Cases per Capita', ascending = False).reset_index()

In [186]:
covid_by_pop.head()

,level_0,index,Country,NDeaths,NCases,Population,Cases per Capita
0,169,179,San Marino,42,742,33944,1237.332076
1,4,162,Peru,31369,762865,33050211,949.131611
2,33,19,Belgium,9948,102201,11598451,857.700740
3,157,4,Andorra,53,1564,77287,685.755690
4,8,195,Spain,30495,640040,46757980,652.188140


In [187]:
fig = px.choropleth(covid_by_pop, locations='Country', color='Cases per Capita',
                        locationmode='country names',
                           color_continuous_scale="Tealgrn",
                           range_color=(10, 600),
                           scope="world",
                           title = 'Worldwide recorded Covid deaths per Capita',
                           labels={'NDeaths':'Total Deaths'}
                          )
fig.show()